In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.mft import Mft
from checklist.inv_dir import Inv, Dir
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.perturb import Perturb


In [15]:
from checklist.pred_wrapper import PredictorWrapper

import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
sentiment = model_wrapper.ModelWrapper()
wrapped_pp = PredictorWrapper.wrap_softmax(sentiment.predict_proba)


In [3]:
editor = checklist.editor.Editor()

In [4]:
import csv
r = csv.DictReader(open('/home/marcotcr/datasets/airline/Tweets.csv'))
labels = []
confs = []
airlines = []
tdata = []
reasons = []
for row in r:
    sentiment, conf, airline, text = row['airline_sentiment'], row['airline_sentiment_confidence'], row['airline'], row['text']
    labels.append(sentiment)
    confs.append(conf)
    airlines.append(airline)
    tdata.append(text)
    reasons.append(row['negativereason'])

mapping = {'negative': 0, 'positive': 2, 'neutral': 1}
labels = np.array([mapping[x] for x in labels]).astype(int)

In [6]:
nlp = spacy.load('en_core_web_sm')

In [8]:
sentences = tdata
parsed_data = list(nlp.pipe(sentences))

In [23]:
def new_pp(data):
    margin_neutral = 1/3.
    mn = margin_neutral / 2.
    pr = wrapped_pp(data)[1][:, 1]
    pp = np.zeros((pr.shape[0], 3))
    neg = pr < 0.5 - mn
    pp[neg, 0] = 1 - pr[neg]
    pp[neg, 2] = pr[neg]
    pos = pr > 0.5 + mn
    pp[pos, 0] = 1 - pr[pos]
    pp[pos, 2] = pr[pos]
    neutral_pos = (pr >= 0.5) * (pr < 0.5 + mn)
    pp[neutral_pos, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_pos] - 0.5)
    pp[neutral_pos, 2] = 1 - pp[neutral_pos, 1]
    neutral_neg = (pr < 0.5) * (pr > 0.5 - mn)
    pp[neutral_neg, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_neg] - 0.5)
    pp[neutral_neg, 0] = 1 - pp[neutral_neg, 1]
    preds = np.argmax(pp, axis=1)
    return preds, pp
    

## Aspect: Vocabulary

### MFTs

In [26]:
air_noun = ['flight', 'seat', 'pilot', 'staff', 'service', 'customer service', 'aircraft', 'plane', 'food', 'cabin crew', 'company', 'airline', 'crew']
editor.add_lexicon('air_noun', air_noun)

In [35]:
print(', '.join(editor.suggest('It was {a:bert} {air_noun}.')[:40]))

good, great, excellent, amazing, new, terrible, awful, old, interesting, unusual, small, incredible, extraordinary, expensive, important, experimental, bad, awesome, big, ordinary, wonderful, huge, fantastic, American, nice, enormous, exceptional, odd, large, unfamiliar, remarkable, professional, empty, skeleton, easy, open, happy, young, outstanding, exemplary


In [531]:
pos_adj = ['good', 'great', 'excellent', 'amazing', 'extraordinary', 'beautiful', 'fantastic', 'nice', 'incredible', 'exceptional', 'awesome', 'perfect', 'fun', 'happy', 'adorable', 'brilliant', 'exciting', 'sweet', 'wonderful']
neg_adj = ['awful', 'bad', 'horrible', 'tough', 'weird', 'aggressive', 'rough', 'lousy', 'unhappy', 'average', 'difficult', 'poor', 'sad', 'frustrating', 'hard', 'lame', 'nasty', 'annoying', 'boring', 'creepy', 'dreadful', 'ridiculous', 'terrible', 'ugly', 'unpleasant']
neutral_adj = ['American', 'international',  'commercial', 'British', 'private', 'Italian', 'Indian', 'Australian', 'Israeli', ]
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )
editor.add_lexicon('neutral_adj', neutral_adj, overwrite=True)

In [47]:
# air_noun

In [54]:
# editor.tg.unmask('I <mask> the food.')

In [57]:
print(', '.join(editor.suggest('I really {bert} the {air_noun}.')[:200]))
print()
print(', '.join(editor.suggest('I {bert} the {air_noun}.')[:50]))

liked, enjoyed, like, appreciate, appreciated, enjoy, loved, love, miss, missed, recommend, wanted, got, needed, likes, prefer, hate, need, admired, want, admire, value, enjoying, enjoys, dislike, dig, use, trust, respect, respected, liking, did, used, get, understand, found, adore, underestimated, preferred, was, valued, feel, have, ,, tried, helped, took, mean, regret, disliked, praised, noticed, dug, hated, loves, had, compliment, remember, cherish, do, rate, about, LOVE, felt, saw, supported, support, applaud, commend, thank, bought, treasure, improved, see, understood, welcome, hit, left, is, beat, all, impressed, know, think, thought, for, believe, thanks, trusted, credit, underestimate, help, take, met, tested, wish, experienced, lost, recommended, blame, owe, leave, in, reviewed, picked, welcomed, made, Love, packed, chose, misses, finished, values, received, delivered, praise, embrace, upgraded, consider, to, changed, worked, into, started, considered, envy, improve, try, fanc

In [289]:
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love',  'recommend', 'admire', 'value', 'welcome']
neg_verb_present = ['hate', 'dislike', 'regret',  'abhor', 'dread', 'despise' ]
neutral_verb_present = ['see', 'find']
pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
neg_verb_past = ['hated', 'disliked', 'regretted',  'abhorred', 'dreaded', 'despised']
neutral_verb_past = ['saw', 'found']
editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

Individual words

In [64]:
test = Mft(pos_adj + pos_verb_present + pos_verb_past, labels=2)
test.run(new_pp)
test.summary(n=3)

In [69]:
test = Mft(neg_adj + neg_verb_present + neg_verb_past, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 38 examples
Test cases:     38
Failure rate:   5.3%

Example fails:
2 (0.8) tough
2 (1.0) aggressive


In [70]:
test = Mft(neutral_adj + neutral_verb_present + neutral_verb_past, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 12 examples
Test cases:     12
Failure rate:   100.0%

Example fails:
2 (1.0) British
2 (1.0) American
2 (1.0) found


Words in context

In [93]:
len(data), len(labels)

(3510, 4992)

In [102]:
# editor.template('I {pos_verb_present} {bert} {air_noun}.')

In [557]:
data = editor.template('{it} {air_noun} {be} {pos_adj}.', it=['The', 'This', 'That'], be=['is', 'was'])
data += editor.template('{it} {be} {a:pos_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'])
data += editor.template('{i} {pos_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'])
labels = [2] * len(data)
data += editor.template('{it} {air_noun} {be} {neg_adj}.', it=['That', 'This', 'The'], be=['is', 'was'])
data += editor.template('{it} {be} {a:neg_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'])
data += editor.template('{i} {neg_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'])
labels += [0] * (len(data) - len(labels))
test = Mft(data, labels=labels)
# test = Mft(data, labels=2)
test.run(new_pp)
test.summary(n=3)

Predicting 8970 examples
Test cases:      8970
Fails (rate):    169 (1.9%)

Example fails:
2 (1.0) That was an aggressive airline.
2 (1.0) This service was aggressive.
2 (1.0) That was an aggressive plane.


In [558]:
data = editor.template('{it} {air_noun} {be} {neutral_adj}.', it=['That', 'This', 'The'], be=['is', 'was'])
data += editor.template('{it} {be} {a:neutral_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'])
data += editor.template('{i} {neutral_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'])
test = Mft(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 1716 examples
Test cases:      1716
Fails (rate):    1626 (94.8%)

Example fails:
0 (1.0) It airline was Indian.
2 (1.0) The seat was private.
2 (1.0) This food was international.


### Intensifiers and reducers

In [110]:
print(' , '.join(editor.suggest('{it} {be} {a:bert} {pos_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'])[:50]))

very , really , absolutely , truly , extremely , quite , most , pretty , incredibly , just , almost , actually , ... , amazingly , especially , exceptionally , surprisingly , unbelievably , equally , utterly , extraordinarily , absolute , overall , otherwise , damn , unusually , obviously , simply , exceedingly , generally , real , a , entirely , overwhelmingly , undeniably , amazing , rather , already , totally , increasingly , altogether , unexpectedly , … , particularly , all , insanely , incredible , fairly , actual , apparently


In [112]:
intens_adj = ['very', 'really', 'absolutely', 'truly', 'extremely', 'quite', 'incredibly', 'amazingly', 'especially', 'exceptionally', 'unbelievably', 'utterly', 'exceedingly', 'rather', 'totally', 'particularly']

In [195]:
print(', '.join(editor.suggest('{i} {bert} {pos_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'])[:100]))

I, really, always, also, highly, definitely, all, personally, truly, certainly, actually, absolutely, still, just, thoroughly, very, greatly, especially, particularly, much, we, quite, so, totally, generally, clearly, seriously, never, 'd, genuinely, rather, honestly, do, people, strongly, both, sure, obviously, did, you, fully, REALLY, ly, sincerely, only, most, guys, already, initely, would, completely, deeply, 've, even, Really, and, simply, ,, too, have, they, will, kids, 's, 'll, mostly, can, should, immediately, must, again, extremely, had, ..., now, least, he, dearly, might, to, everyone, fucking, could, kinda, 't, probably, usually, immensely, may, family, hardly, o, does, specifically, many, students, boys, secretly, more, everybody


In [198]:
intens_verb = [ 'really', 'absolutely', 'truly', 'extremely',  'especially',  'utterly',  'totally', 'particularly', 'highly', 'definitely', 'certainly', 'genuinely', 'honestly', 'strongly', 'sure', 'sincerely']

In [159]:
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.1)
non_neutral_pred = lambda pred, *args, **kwargs: pred != 1
monotonic_label = Expect.slice_pairwise(monotonic_label, non_neutral_pred)

In [203]:
data = editor.template(['{it} {be} {a:pos_adj} {air_noun}.', '{it} {be} {a:intens} {pos_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500)
data += editor.template(['{i} {pos_verb} {the} {air_noun}.', '{i} {intens} {pos_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500)
data += editor.template(['{it} {be} {a:neg_adj} {air_noun}.', '{it} {be} {a:intens} {neg_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500)
data += editor.template(['{i} {neg_verb} {the} {air_noun}.', '{i} {intens} {neg_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500)
test = Dir(data, monotonic_label)
test.run(new_pp)
test.set_monotonic_print(increasing=True)
test.summary(3)
# test = Mft(data, labels=labels)
# test = Mft(data, labels=2)
# test.run(new_pp)
# test.summary(n=3)

Predicting 4000 examples
Test cases:      2000
After filtering: 1996 (99.8%)
Fails (rate):    26 (1.3%)

Example fails:
0 (1.0) This is a commercial plane.
2 (1.0) This is an exceedingly commercial plane.

0 (0.9) It is a commercial aircraft.
2 (1.0) It is a really commercial aircraft.

0 (1.0) This was a creepy seat.
2 (1.0) This was an amazingly creepy seat.



In [162]:
reducer_adj = ['somewhat', 'kinda', 'mostly', 'probably', 'generally', 'reasonably', 'little', 'bit', 'slightly']

In [163]:
monotonic_label_down = Expect.monotonic(increasing=False, tolerance=0.1)
monotonic_label_down = Expect.slice_pairwise(monotonic_label_down, non_neutral_pred)

In [185]:
data = editor.template(['{it} {be} {a:pos_adj} {air_noun}.', '{it} {be} {a:red} {pos_adj} {air_noun}.'] , red=reducer_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=1000)
data += editor.template(['{it} {be} {a:neg_adj} {air_noun}.', '{it} {be} {a:red} {neg_adj} {air_noun}.'] , red=reducer_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=1000)
test = Dir(data, monotonic_label_down)
test.run(new_pp)
test.set_monotonic_print(increasing=False)
test.summary(3)
# test = Mft(data, labels=labels)
# test = Mft(data, labels=2)
# test.run(new_pp)
# test.summary(n=3)

Predicting 4000 examples
Test cases:     2000
Cases filtered out: 1987 (99.3%)
Failure rate:   30.8%

Example fails:
0 (0.9) It was an average plane.
0 (1.0) It was a kinda average plane.

0 (0.9) It is an average flight.
0 (1.0) It is a probably average flight.

0 (0.9) It is a creepy food.
0 (1.0) It is a probably creepy food.



### Invariance: change neutral words

In [265]:
neutral_words = set(
    ['.', 'the', 'The', ',', 'a', 'A', 'and', 'of', 'to', 'it', 'that', 'in',
     'this', 'for',  'you', 'there', 'or', 'an', 'by', 'about', 'flight', 'my',
     'in', 'of', 'have', 'with', 'was', 'at', 'it', 'get', 'from', 'this', 'Flight', 'plane'
    ])
forbidden = set(['No', 'no', 'Not', 'not', 'Nothing', 'nothing', 'without'] + pos_adj + neg_adj + pos_verb_present + pos_verb_past + neg_verb_present + neg_verb_past)
def change_neutral(d):
#     return d.text
    examples = []
    words_in = [x for x in d.capitalize().split() if x in neutral_words]
    if not words_in:
        return None
    for w in words_in:
        examples.extend([x[1] for x in editor.suggest_replace(d, w, beam_size=5, words_and_sentences=True) if x[0] not in forbidden])
    if examples:
        idxs = np.random.choice(len(examples), min(len(examples), 10), replace=False)
        return [examples[i] for i in idxs]
# Perturb.perturb(parsed_data[:5], perturb)

In [269]:
data = Perturb.perturb(sentences, change_neutral, nsamples=500)

In [273]:
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 5009 examples
Test cases:      500
Fails (rate):    64 (12.8%)

Example fails:
0 (1.0) @SouthwestAir how do I get a companion pass
1 (0.9) @SouthwestAir how do I add a companion pass

2 (0.9) @united well perhaps you could highlight where that is stated.  We travel between Canada &amp; USA on United with only Nexus
1 (0.8) @united well perhaps they could highlight where that is stated.  We travel between Canada &amp; USA on United with only Nexus
1 (0.9) @united well perhaps someone could highlight where that is stated.  We travel between Canada &amp; USA on United with only Nexus

2 (0.9) @JetBlue sadly, no! I have the app, but it also is experiencing difficulties. The flight information boards are keeping me updated.
0 (0.7) @JetBlue sadly, no! I checked the app, but it also is experiencing difficulties. The flight information boards are keeping me updated.
0 (0.7) @JetBlue sadly, no! I have the app, but it also is experiencing difficulties. The airline information boards 

### Add negative phrases

In [296]:
positive = editor.template('I {pos_verb_present} you.')
positive += editor.template('You are {pos_adj}.')
positive += ['I would fly with you again.']
positive.remove('You are happy.')
negative = editor.template('I {neg_verb_present} you.')
negative += editor.template('You are {neg_adj}.')
negative += ['Never flying with you again.']
def add_phrase_function(phrases):
    def pert(d):
        while d[-1].pos_ == 'PUNCT':
            d = d[:-1]
        d = d.text
        ret = [d + '. ' + x for x in phrases]
        idx = np.random.choice(len(ret), 10, replace=False)
        ret = [ret[i] for i in idx]
        return ret
    return pert

# perturbed = PerturbFactory.perturb_key(small, 'sentence', add_phrase_function(positive))
# test = mltest.Test(perturbed, expectation_fn = mon_increasing)
# r = test.run(model.predict_and_confidence, is_binary=False, n=500)
# r.summary(5, format_fn=format_perturb_neg)


In [301]:
monotonic_1 = Expect.monotonic(label=2, increasing=True, tolerance=0.1)
monotonic_1_down = Expect.monotonic(label=2, increasing=False, tolerance=0.1)

In [300]:
data = Perturb.perturb(parsed_data, add_phrase_function(positive), nsamples=500)
test = Dir(data, monotonic_1)
test.run(new_pp, overwrite=True)
test.set_monotonic_print(label=2, increasing=True)
test.summary(3)

Predicting 5500 examples
Test cases:      500
After filtering: 150 (30.0%)
Fails (rate):    0 (0.0%)


In [302]:
data = Perturb.perturb(parsed_data, add_phrase_function(negative), nsamples=500)
test = Dir(data, monotonic_1_down)
test.run(new_pp, overwrite=True)
test.set_monotonic_print(label=2, increasing=False)
test.summary(3)

Predicting 5500 examples
Test cases:      500
After filtering: 370 (74.0%)
Fails (rate):    83 (22.4%)

Example fails:
0 (0.8) @USAirways Nice try.  Doesn't apply to Philly.
2 (1.0) @USAirways Nice try.  Doesn't apply to Philly. You are tough.
0 (0.8) @USAirways Nice try.  Doesn't apply to Philly. You are aggressive.

0 (0.8) @united maybe one day you'll be the one quoted on http://t.co/mJkpgVXmPC
2 (1.0) @united maybe one day you'll be the one quoted on http://t.co/mJkpgVXmPC. You are aggressive.

0 (1.0) @SouthwestAir schedule is open through the end of October, got Columbus day wknd in New England for ~50K points for 4! Check your calendar!
2 (0.9) @SouthwestAir schedule is open through the end of October, got Columbus day wknd in New England for ~50K points for 4! Check your calendar. You are aggressive.
0 (1.0) @SouthwestAir schedule is open through the end of October, got Columbus day wknd in New England for ~50K points for 4! Check your calendar. You are tough.



## Aspect: robustness
### Invariance: adding irrelevant stuff before and after.


In [303]:
import string
def random_string(n):
    return ''.join(np.random.choice([x for x in string.ascii_letters + string.digits], n))
def random_url(n=6):
    return 'https://t.co/%s' % random_string(n)
def random_handle(n=6):
    return '@%s' % random_string(n)

# data['sentence']

def add_irrelevant(sentence):
    urls_and_handles = [random_url(n=6) for _ in range(5)] + [random_handle() for _ in range(5)]
    irrelevant_before = ['@airline '] + urls_and_handles
    irrelevant_after = urls_and_handles 
    rets = ['%s %s' % (x, sentence) for x in irrelevant_before ]
    rets += ['%s %s' % (sentence, x) for x in irrelevant_after]
    return rets


In [304]:
data = Perturb.perturb(sentences, add_irrelevant, nsamples=500)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 11000 examples
Test cases:      500
Fails (rate):    56 (11.2%)

Example fails:
0 (0.9) @united well. As of yet, our checked bag has already vanished and we haven't left the airport yet.
2 (0.7) @united well. As of yet, our checked bag has already vanished and we haven't left the airport yet. @lUVsAS
2 (0.9) @united well. As of yet, our checked bag has already vanished and we haven't left the airport yet. @N6GErs

1 (0.7) @JetBlue's new CEO seeks the right balance to please passengers and Wall ... - Daily Journal http://t.co/9bzqZQx8DC
2 (0.8) @airline  @JetBlue's new CEO seeks the right balance to please passengers and Wall ... - Daily Journal http://t.co/9bzqZQx8DC
0 (1.0) https://t.co/Y7VByc @JetBlue's new CEO seeks the right balance to please passengers and Wall ... - Daily Journal http://t.co/9bzqZQx8DC

2 (1.0) @AmericanAir come on I just want to go home I can't miss another day of work #stuckinmemphis #texasisclosed
1 (0.5) @LwpRRX @AmericanAir come on I just want to 

### punctuation, contractions

In [312]:
data = Perturb.perturb(parsed_data, Perturb.punctuation, nsamples=500)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 1162 examples
Test cases:      500
Fails (rate):    15 (3.0%)

Example fails:
2 (0.7) @SouthwestAir F5R3ZZ
0 (0.8) @SouthwestAir F5R3ZZ.

0 (0.8) @USAirways it still says that I can't check into my flight because the information is incorrect but everything is entered correctly
1 (0.6) @USAirways it still says that I can't check into my flight because the information is incorrect but everything is entered correctly.

0 (0.9) @usairways I get extended hold times due to the weather...but 2-1/2 hours on hold (and counting). HELP!!
2 (0.8) @usairways I get extended hold times due to the weather...but 2-1/2 hours on hold (and counting). HELP.



In [327]:
data = Perturb.perturb(sentences, Perturb.contractions, nsamples=1000)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 2087 examples
Test cases:      1000
Fails (rate):    29 (2.9%)

Example fails:
2 (0.7) @SouthwestAir    How's life @ the NOC?
0 (0.9) @SouthwestAir    How is life @ the NOC?

2 (0.9) @AmericanAir it's flight 5348
1 (0.6) @AmericanAir it is flight 5348

1 (0.9) @USAirways any update on this flight?It's more than three hours delayed. Thank you.
2 (0.7) @USAirways any update on this flight?It is more than three hours delayed. Thank you.



## Aspect: NER

In [457]:

data = Perturb.perturb(parsed_data, Perturb.change_names, nsamples=1000)

In [440]:
data = Perturb.perturb(parsed_data, Perturb.change_names, nsamples=1000)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 3641 examples
Test cases:      331
Fails (rate):    25 (7.6%)

Example fails:
0 (0.8) @USAirways owes Tammy from the Winston-Salem call center for keeping me as a customer!
2 (0.8) @USAirways owes Chelsea from the Winston-Salem call center for keeping me as a customer!
2 (0.8) @USAirways owes Hannah from the Winston-Salem call center for keeping me as a customer!

0 (0.9) @united Hey so many time changes for UA 1534. We going tonight or what? MIA - EWR
2 (0.7) @united Hey so many time changes for UA 1534. We going tonight or what? Amanda Green
1 (0.8) @united Hey so many time changes for UA 1534. We going tonight or what? Stephanie Robinson

2 (0.9) @JetBlue your employee Charles cave at the gate at MSY went above and beyond to help try to help me find my glasses. Thought u should know
0 (0.9) @JetBlue your employee Jesus cave at the gate at MSY went above and beyond to help try to help me find my glasses. Thought u should know



In [441]:
data = Perturb.perturb(parsed_data, Perturb.change_location, nsamples=1000)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 9999 examples
Test cases:      909
Fails (rate):    61 (6.7%)

Example fails:
2 (0.7) @USAirways us2118
My wife in Boston says no snow right now.
1 (0.6) @USAirways us2118
My wife in Greenville says no snow right now.
0 (0.8) @USAirways us2118
My wife in Park Ridge says no snow right now.

1 (0.5) @SouthwestAir to start daily #B737-700 flights from #Columbus OH to #Oakland on 8AUG #avgeek
0 (0.7) @SouthwestAir to start daily #B737-700 flights from #Columbus OH to #Warner Robins on 8AUG #avgeek
0 (0.7) @SouthwestAir to start daily #B737-700 flights from #Columbus OH to #Richmond on 8AUG #avgeek

2 (0.9) @SouthwestAir we have to stay in Chicago overnight and meet up with our bags there. I hope they stay put and don't get put on a flight.
1 (0.5) @SouthwestAir we have to stay in Cranston overnight and meet up with our bags there. I hope they stay put and don't get put on a flight.
1 (0.7) @SouthwestAir we have to stay in Dunwoody overnight and meet up with our bags there. I hop

In [467]:
data = Perturb.perturb(parsed_data, Perturb.change_number, nsamples=1000)
test = Inv(data)
test.run(new_pp)
test.summary(3)

Predicting 11000 examples
Test cases:      1000
Fails (rate):    21 (2.1%)

Example fails:
1 (0.9) @americanair the best is your 800 message saying to use website and your website is saying you need to call.  If you don't answer, #hardtodo
2 (0.9) @americanair the best is your 704 message saying to use website and your website is saying you need to call.  If you don't answer, #hardtodo
2 (1.0) @americanair the best is your 670 message saying to use website and your website is saying you need to call.  If you don't answer, #hardtodo

0 (0.9) @VirginAmerica save some for 871 tomorrow AM!
1 (0.6) @VirginAmerica save some for 826 tomorrow AM!

0 (0.7) @USAirways   my miles will expire on 2/29 and it could take someone 10 days to respond...I have over 150000 miles that I do not lose i❤usair
1 (0.6) @USAirways   my miles will expire on 2/29 and it could take someone 10 days to respond...I have over 134350 miles that I do not lose i❤usair
1 (0.6) @USAirways   my miles will expire on 2/29 and 

## Aspect: temporal awareness

In [336]:
editor.template('{neg_verb_present}')

['hate', 'dislike', 'regret', 'abhor', 'dread', 'despise']

In [337]:
change = ['but', 'even though', 'although', '']
data = editor.template(['I used to think this airline was {neg_adj}, {change} now I think it is {pos_adj}.',
                                 'I think this airline is {pos_adj}, {change} I used to think it was {neg_adj}.',
                                 'In the past I thought this airline was {neg_adj}, {change} now I think it is {pos_adj}.',
                                 'I think this airline is {pos_adj}, {change} in the past I thought it was {neg_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=500)
data += editor.template(['I used to {neg_verb_present} this airline, {change} now I {pos_verb_present} it.',
                                 'I {pos_verb_present} this airline, {change} I used to {neg_verb_present} it.',
                                 'In the past I would {neg_verb_present} this airline, {change} now I {pos_verb} it.',
                                 'I {pos_verb_present} this airline, {change} in the past I would {neg_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=500)
labels = [2] * len(data)

data += editor.template(['I used to think this airline was {pos_adj}, {change} now I think it is {neg_adj}.',
                                 'I think this airline is {neg_adj}, {change} I used to think it was {pos_adj}.',
                                 'In the past I thought this airline was {pos_adj}, {change} now I think it is {neg_adj}.',
                                 'I think this airline is {neg_adj}, {change} in the past I thought it was {pos_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=500)
data += editor.template(['I used to {pos_verb_present} this airline, {change} now I {neg_verb_present} it.',
                                 'I {neg_verb_present} this airline, {change} I used to {pos_verb_present} it.',
                                 'In the past I would {pos_verb_present} this airline, {change} now I {neg_verb_present} it.',
                                 'I {neg_verb_present} this airline, {change} in the past I would {pos_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=500)
labels += [0] * (len(data) - len(labels))
test = Mft(data, labels=labels)
test.run(new_pp)
test.summary(n=3)



Predicting 8000 examples
Test cases:      8000
Fails (rate):    1531 (19.1%)

Example fails:
0 (0.9) I think this airline is great, but I used to think it was difficult.
2 (1.0) I abhor this airline, but in the past I would love it.
2 (1.0) I used to admire this airline, even though now I despise it.


used to should reduce

In [352]:
data = editor.template(['{it} {be} {a:adj} {air_noun}.', 'I used to think {it} {be} {a:adj} {air_noun}.'], it=['it', 'this', 'that'], be=['is', 'was'], adj=editor.lexicons['pos_adj'] + editor.lexicons['neg_adj'])
data += editor.template(['{i} {verb} {the} {air_noun}.', '{i} used to {verb} {the} {air_noun}.'], i=['I', 'We'], the=['this', 'that', 'the'], verb=editor.lexicons['pos_verb_present'] + editor.lexicons['neg_verb_present'])
test = Dir(data, monotonic_label_down)
# test = Mft(data, labels=2)
test.run(new_pp)
test.summary(n=3)

Predicting 9204 examples
Test cases:      4602
After filtering: 27 (0.6%)
Fails (rate):    14 (51.9%)

Example fails:
0 (0.8) this was a commercial flight.
0 (1.0) I used to think this was a commercial flight.

0 (0.9) it is an average food.
0 (1.0) I used to think it is an average food.

0 (0.9) that was a commercial company.
0 (1.0) I used to think that was a commercial company.



### Aspect:fairness

In [509]:
editor.lexicons.keys()

dict_keys(['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'air_noun', 'neutral_noun', 'pos_adj', 'neg_adj', 'pos_verb_present', 'neg_verb_present', 'neutral_verb_present', 'pos_verb_past', 'neg_verb_past', 'neutral_verb_past', 'neutral_adj', 'pos_verb', 'neg_verb', 'neutral_verb'])

In [511]:
editor.template('{a:religion_adj}'),

(['a Christian',
  'a Protestant',
  'a Roman Catholic',
  'an Eastern Orthodox',
  'an Anglican',
  'a Jew',
  'an Orthodox Jew',
  'a Muslim',
  'a Sunni',
  "a Shi'a",
  'an Ahmadiyya',
  'a Buddhist',
  'a Zoroastrian',
  'a Hindu',
  'a Sikh',
  'a Shinto',
  "a Baha'i",
  'a Taoist',
  'a Confucian',
  'a Jain',
  'an Atheist',
  'an Agnostic'],)

In [517]:
protected = {
    'race': ['a black','a hispanic', 'a white', 'an asian'],
    'sexual': editor.template('{a:sexual_adj}'),
    'religion': editor.template('{a:religion_adj}'),
    'nationality': editor.template('{a:nationality}')[:20],
}

In [521]:
for p, vals in protected.items():
    print(p)
    d = editor.template(['{male} is %s {bert}.' % r for r in vals], return_maps=False, nsamples=100)
    d += editor.template(['{female} is %s {bert}.' % r for r in vals], return_maps=False, nsamples=100)
    test = Inv(d, threshold=0.1)
    test.run(new_pp)
    test.summary(n=3)
    print()
    preds = np.array(test.results.preds)
    for i, x in enumerate(vals):
        print('%.2f %s' % (preds[:, i].mean(), vals[i]))
    print()
    print()
    print('-------------------------')

race
Predicting 800 examples
Test cases:      200
Fails (rate):    182 (91.0%)

Example fails:
0 (0.9) Leslie is a black girl.
2 (1.0) Leslie is a hispanic girl.
2 (1.0) Leslie is an asian girl.

0 (0.9) Caroline is a black American.
2 (1.0) Caroline is an asian American.
2 (1.0) Caroline is a hispanic American.

0 (0.7) Victoria is a black boy.
2 (1.0) Victoria is a hispanic boy.
2 (1.0) Victoria is an asian boy.


0.30 a black
1.96 a hispanic
0.51 a white
2.00 an asian


-------------------------
sexual
Predicting 2800 examples
Test cases:      200
Fails (rate):    200 (100.0%)

Example fails:
2 (1.0) Connor is an asexual woman.
0 (0.8) Connor is a lesbian woman.
0 (0.8) Connor is a trans woman.

2 (1.0) Danielle is an asexual feminist.
0 (0.7) Danielle is a transgender feminist.
0 (0.7) Danielle is a heterosexual feminist.

2 (1.0) Diana is an asexual person.
1 (0.5) Diana is a cisgender person.
0 (0.7) Diana is a bisexual person.


1.85 an asexual
1.41 a bisexual
0.64 a heterosexua


### Aspect: Negation

Simple templates:

In [547]:
data = editor.template('{it} {air_noun} {nt} {pos_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'])
data += editor.template('{it} {benot} {a:pos_adj} {air_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'])
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
data += editor.template('{neg} {pos_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'])
data += editor.template('No one {pos_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'])
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 2964 examples
Test cases:      2964
Fails (rate):    0 (0.0%)


In [559]:
data = editor.template('{it} {air_noun} {nt} {neg_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'])
data += editor.template('{it} {benot} {a:neg_adj} {air_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'])
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
data += editor.template('{neg} {neg_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'])
data += editor.template('No one {neg_verb_present}s {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'])
# expectation: prediction is not 0
is_not_0 = lambda x, pred, *args: pred != 0
test = Mft(data, Expect.single(is_not_0))
test.run(new_pp)
test.summary(n=3)


Predicting 7254 examples
Test cases:      7254
Fails (rate):    1255 (17.3%)

Example fails:
0 (1.0) No one hates the pilot.
0 (0.9) That seat isn't aggressive.
0 (1.0) I don't abhor that aircraft.


In [560]:
data = editor.template('{it} {air_noun} {nt} {neutral_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'])
data += editor.template('{it} {benot} {a:neutral_adj} {air_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'])
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
data += editor.template('{neg} {neutral_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'])
test = Mft(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 2496 examples
Test cases:      2496
Fails (rate):    2466 (98.8%)

Example fails:
0 (1.0) That is not an Italian aircraft.
0 (0.8) It isn't an Israeli seat.
0 (1.0) This plane isn't private.


In [ ]:
Different templates:

In [561]:
data = editor.template('I thought {it} {air_noun} would be {pos_adj}, but it {neg}.', neg=['was not', 'wasn\'t'], it=['this', 'that', 'the'], nt=['is not', 'isn\'t'])
data += editor.template('I thought I would {pos_verb_present} {the} {air_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that', 'the'])
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 2106 examples
Test cases:      2106
Fails (rate):    32 (1.5%)

Example fails:
2 (0.8) I thought I would admire this aircraft, but I did not.
1 (0.7) I thought I would admire this seat, but I did not.
2 (0.8) I thought I would love that plane, but I did not.


In [562]:
data = editor.template('I thought {it} {air_noun} would be {neg_adj}, but it {neg}.', neg=['was not', 'wasn\'t'], it=['this', 'that', 'the'], nt=['is not', 'isn\'t'])
data += editor.template('I thought I would {neg_verb_present} {the} {air_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that', 'the'])
# expectation: prediction is not 0
test = Mft(data, Expect.single(is_not_0))
test.run(new_pp)
test.summary(n=3)


Predicting 2418 examples
Test cases:      2418
Fails (rate):    2082 (86.1%)

Example fails:
0 (0.9) I thought the staff would be ugly, but it wasn't.
0 (1.0) I thought that flight would be frustrating, but it was not.
0 (1.0) I thought the plane would be dreadful, but it wasn't.


In [563]:
data = editor.template('I thought {it} {air_noun} would be {neutral_adj}, but it {neg}.', neg=['was not', 'wasn\'t'], it=['this', 'that', 'the'], nt=['is not', 'isn\'t'])
data += editor.template('I thought I would {neutral_verb_present} {the} {air_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that', 'the'])
# expectation: prediction is not 0
test = Mft(data, labels=1)
test.run(new_pp)
test.summary(n=3)


Predicting 858 examples
Test cases:      858
Fails (rate):    844 (98.4%)

Example fails:
0 (1.0) I thought this food would be Australian, but it wasn't.
0 (1.0) I thought this flight would be American, but it was not.
2 (0.9) I thought I would see this flight, but I did not.


Harder: negation with neutral in the middle

In [570]:
neutral =['that I am from Brazil', 'my history with airplanes', 'all that I\'ve seen over the years', 'the time that I\'ve been flying', 'it\'s a Tuesday']
data = editor.template('{neg}, given {neutral}, that {it} {air_noun} {be} {pos_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'])
data += editor.template('{neg}, given {neutral}, that {it} {be} {a:pos_adj} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'])
data += editor.template('{neg}, given {neutral}, that {i} {pos_verb_present} {the} {air_noun}.',neutral=neutral,  neg=neg, i=['I', 'we'], the=['this', 'that', 'the'])
data = list(np.random.choice(data, 1000, replace=False))
test = Mft(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    706 (70.6%)

Example fails:
2 (1.0) I wouldn't say, given the time that I've been flying, that that was a sweet customer service.
1 (0.9) I wouldn't say, given that I am from Brazil, that this was a happy plane.
2 (1.0) I can't say, given my history with airplanes, that this flight is fantastic.


In [572]:
neutral =['that I am from Brazil', 'my history with airplanes', 'all that I\'ve seen over the years', 'the time that I\'ve been flying', 'it\'s a Tuesday']
data = editor.template('{neg}, given {neutral}, that {it} {air_noun} {be} {neg_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'])
data += editor.template('{neg}, given {neutral}, that {it} {be} {a:neg_adj} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'])
data += editor.template('{neg}, given {neutral}, that {i} {neg_verb_present} {the} {air_noun}.',neutral=neutral,  neg=neg, i=['I', 'we'], the=['this', 'that', 'the'])
data = list(np.random.choice(data, 1000, replace=False))
test = Mft(data, Expect.single(is_not_0))
test.run(new_pp)
test.summary(n=3)


Predicting 1000 examples
Test cases:      1000
Fails (rate):    992 (99.2%)

Example fails:
0 (1.0) I wouldn't say, given it's a Tuesday, that this was a lame airline.
0 (1.0) I can't say, given that I am from Brazil, that that was a poor airline.
0 (1.0) I can't say, given it's a Tuesday, that that was a frustrating company.


In [573]:
neutral =['that I am from Brazil', 'my history with airplanes', 'all that I\'ve seen over the years', 'the time that I\'ve been flying', 'it\'s a Tuesday']
data = editor.template('{neg}, given {neutral}, that {it} {air_noun} {be} {neutral_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'])
data += editor.template('{neg}, given {neutral}, that {it} {be} {a:neutral_adj} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'])
data += editor.template('{neg}, given {neutral}, that {i} {neutral_verb_present} {the} {air_noun}.',neutral=neutral,  neg=neg, i=['I', 'we'], the=['this', 'that', 'the'])
data = list(np.random.choice(data, 1000, replace=False))
test = Mft(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    984 (98.4%)

Example fails:
0 (1.0) I don't think, given all that I've seen over the years, that this service was Australian.
0 (1.0) I wouldn't say, given the time that I've been flying, that this seat was Australian.
0 (1.0) I can't say, given my history with airplanes, that this is a commercial staff.



### Aspect: SRL

my opinion is more important than others

In [576]:
change = [' but', '']
templates = ['Some people think you are {neg_adj},{change} I think you are {pos_adj}.',
             'I think you are {pos_adj},{change} some people think you are {neg_adj}.',
             'I had heard you were {neg_adj},{change} I think you are {pos_adj}.',
             'I think you are {pos_adj},{change} I had heard you were {neg_adj}.',
             ]
data = editor.template(templates, change=change, unroll=True)
templates = ['{others} {neg_verb_present} you,{change} I {pos_verb_present} you.',
             'I {pos_verb_present} you,{change} {others} {neg_verb_present} you.',
            ]
others = ['some people', 'my parents', 'my friends', 'people']
data += editor.template(templates, others=others, change=change, unroll=True)
labels = [2] * len(data)

change = [' but', '']
templates = ['Some people think you are {pos_adj},{change} I think you are {neg_adj}.',
             'I think you are {neg_adj},{change} some people think you are {pos_adj}.',
             'I had heard you were {pos_adj},{change} I think you are {neg_adj}.',
             'I think you are {neg_adj},{change} I had heard you were {pos_adj}.',
             ]
data += editor.template(templates, change=change, unroll=True)
templates = ['{others} {pos_verb_present} you,{change} I {neg_verb_present} you.',
             'I {neg_verb_present} you,{change} {others} {pos_verb_present} you.',
            ]
others = ['some people', 'my parents', 'my friends', 'people']
data += editor.template(templates, others=others, change=change, unroll=True)
labels += [0] * (len(data) - len(labels))
test = Mft(data, labels=labels)
test.run(new_pp)
test.summary(n=3)

Predicting 9136 examples
Test cases:      9136
Fails (rate):    3374 (36.9%)

Example fails:
0 (1.0) I think you are incredible, but some people think you are lame.
0 (1.0) I think you are sweet, but some people think you are bad.
2 (1.0) I think you are horrible, but some people think you are awesome.


q & a form: yes

In [588]:
temp = editor.template('Do I think {it} {air_noun} {be} {pos_adj}?', it=['that', 'this', 'the'], be=['is', 'was'])
temp += editor.template('Do I think {it} {be} {a:pos_adj} {air_noun}?', it=['it', 'this', 'that'], be=['is', 'was'])
temp += editor.template('Did {i} {pos_verb_present} {the} {air_noun}?', i=['I', 'we'], the=['this', 'that', 'the'])
data = [x + ' Yes' for x in temp]
labels = [2] * len(data)
temp2 = editor.template('Do I think {it} {air_noun} {be} {neg_adj}?', it=['that', 'this', 'the'], be=['is', 'was'])
temp2 += editor.template('Do I think {it} {be} {a:neg_adj} {air_noun}?', it=['it', 'this', 'that'], be=['is', 'was'])
temp2 += editor.template('Did {i} {neg_verb_present} {the} {air_noun}?', i=['I', 'we'], the=['this', 'that', 'the'])
data += [x + ' Yes' for x in temp2]
labels += [0] * (len(data) - len(labels))

test = Mft(data, labels=labels)
# test = Mft(data, labels=2)
test.run(new_pp)
test.summary(n=3)

Predicting 7956 examples
Test cases:      7956
Fails (rate):    226 (2.8%)

Example fails:
1 (0.6) Do I think this was a nice staff? Yes
2 (0.9) Did we regret this seat? Yes
0 (1.0) Do I think the cabin crew was nice? Yes


In [590]:
temp3 = editor.template('Do I think {it} {air_noun} {be} {neutral_adj}?', it=['that', 'this', 'the'], be=['is', 'was'])
temp3 += editor.template('Do I think {it} {be} {a:neutral_adj} {air_noun}?', it=['it', 'this', 'that'], be=['is', 'was'])
temp3 += editor.template('Did {i} {neutral_verb_present} {the} {air_noun}?', i=['I', 'we'], the=['this', 'that', 'the'])
data = [x + ' Yes' for x in temp3]
test = Mft(data, labels=1)
# test = Mft(data, labels=2)
test.run(new_pp)
test.summary(n=3)


Predicting 1560 examples
Test cases:      1560
Fails (rate):    1541 (98.8%)

Example fails:
0 (1.0) Do I think that is an Italian food? Yes
0 (1.0) Do I think this company is Indian? Yes
0 (1.0) Do I think that airline was Indian? Yes


In [596]:
data = [x + ' No' for x in temp]
labels = [0] * len(data)
data += [x + ' No' for x in temp2]
labels += [1] * (len(data) - len(labels))

allow_for_neutral = lambda x, pred, _, label, _2 : pred != 0 if label == 1 else pred == label

test = Mft(data, Expect.single(allow_for_neutral), labels=labels)
# test = Mft(data, labels=2)
test.run(new_pp)
test.summary(n=3)

Predicting 7956 examples
Test cases:      7956
Fails (rate):    4371 (54.9%)

Example fails:
0 (1.0) Do I think the service was nasty? No
0 (1.0) Do I think the airline was creepy? No
0 (1.0) Do I think this was a rough service? No


In [595]:
data = [x + ' No' for x in temp3]
test = Mft(data, labels=1)
# test = Mft(data, labels=2)
test.run(new_pp)
test.summary(n=3)

Predicting 1560 examples
Test cases:      1560
Fails (rate):    1560 (100.0%)

Example fails:
0 (1.0) Do I think this plane is international? No
0 (1.0) Do I think that food was Italian? No
0 (1.0) Do I think that crew is American? No
